# Concurrencia y Paralelismo

A medida que una aplicación crece, manejar muchas tareas a la vez se vuelve un desafío. El procesamiento **secuencial** (hacer una cosa a la vez) no es eficiente. Para solucionar esto, usamos dos técnicas: Concurrencia y Paralelismo.

**La Analogía: Un Chef en una Cocina**
* **Secuencial:** El chef prepara un plato de principio a fin (corta, cocina, sirve) y solo entonces empieza con el siguiente plato.
* **Concurrencia:** El chef pone a hervir el agua para la pasta (una tarea de espera). Mientras espera, empieza a cortar los vegetales para la ensalada. **Es un solo chef que gestiona inteligentemente su tiempo de espera**.
* **Paralelismo:** El chef contrata a dos ayudantes. Ahora, **tres chefs (tres núcleos de CPU)** trabajan en tres platos diferentes al mismo tiempo.

## 1. Concurrencia con `threading`

La concurrencia permite que múltiples tareas progresen en periodos de tiempo superpuestos. Es ideal para tareas **"I/O-bound"** (limitadas por Entrada/Salida), donde el programa pasa mucho tiempo **esperando** por una respuesta de la red, un disco duro o una API.

Usamos la librería `threading` para crear "hilos" que gestionan estas tareas en espera.

In [1]:
import threading
import time

def procesar_solicitud(id_solicitud):
    """Simula una tarea que tarda tiempo, como una llamada a una API."""
    print(f"Iniciando procesamiento de la solicitud {id_solicitud}...")
    # time.sleep() simula una operación de espera (I/O)
    time.sleep(2)
    print(f"--> Solicitud {id_solicitud} completada.")

# --- Ejecución Secuencial (para comparar) ---
start_time = time.time()
procesar_solicitud(1)
procesar_solicitud(2)
print(f"Tiempo total secuencial: {time.time() - start_time:.2f} segundos.")

# --- Ejecución Concurrente ---
print("\nIniciando ejecución concurrente...")
hilos = []
start_time = time.time()
for i in range(1, 3):
    # Creamos un hilo para cada solicitud
    hilo = threading.Thread(target=procesar_solicitud, args=(i,))
    hilos.append(hilo)
    # Iniciamos el hilo, que empezará a ejecutarse en segundo plano
    hilo.start()

# Esperamos a que todos los hilos terminen
for hilo in hilos:
    hilo.join()

print(f"Tiempo total concurrente: {time.time() - start_time:.2f} segundos.")

Iniciando procesamiento de la solicitud 1...
--> Solicitud 1 completada.
Iniciando procesamiento de la solicitud 2...
--> Solicitud 2 completada.
Tiempo total secuencial: 4.00 segundos.

Iniciando ejecución concurrente...
Iniciando procesamiento de la solicitud 1...
Iniciando procesamiento de la solicitud 2...
--> Solicitud 1 completada.--> Solicitud 2 completada.

Tiempo total concurrente: 2.00 segundos.


## 2. Paralelismo con `multiprocessing`

El paralelismo ejecuta múltiples tareas **verdaderamente al mismo tiempo**, cada una en un núcleo de CPU diferente. Es ideal para tareas **"CPU-bound"** (limitadas por el procesador), como cálculos matemáticos pesados, simulaciones o procesamiento de grandes volúmenes de datos.

In [ ]:
# Este código debe guardarse en un archivo .py y ejecutarse desde la terminal

import multiprocessing
import time

# Función que realiza un cálculo intensivo
def calcular_cuadrado(n):
    return n * n

if __name__ == "__main__":
    numeros = [1, 2, 3, 4, 5, 6, 7, 8]

    start_time = time.time()

    # Creamos un "pool" de procesos (usará tantos como núcleos de CPU tengas)
    with multiprocessing.Pool() as pool:
        # pool.map() distribuye la lista 'numeros' entre los procesos disponibles
        # y aplica la función 'calcular_cuadrado' a cada elemento en paralelo.
        resultados = pool.map(calcular_cuadrado, numeros)

    print(f"Resultados: {resultados}")
    print(f"Tiempo de ejecución en paralelo: {time.time() - start_time:.4f} segundos.")

## ¿Cuándo Usar Cada Uno?

| Paradigma | Analogía | Tipo de Tarea Ideal | Librería |
| :--- | :--- | :--- | :--- |
| **Concurrencia** | Un solo chef haciendo malabares con varias ollas | Esperas de red, lecturas de archivos (I/O-bound) | `threading`, `asyncio` |
| **Paralelismo** | Varios chefs, cada uno en su propio plato | Cálculos matemáticos, procesamiento de imágenes (CPU-bound) | `multiprocessing` |

In [1]:
import threading

class CuentaBancaria:
    def __init__(self, saldo):
        self.saldo = saldo
        self.lock = threading.RLock()

    def transferir(self, otra_cuenta, cantidad):
        with self.lock:
            self.saldo -= cantidad
            otra_cuenta.depositar(cantidad)

    def depositar(self, cantidad):
        with self.lock:
            self.saldo += cantidad

cuenta1 = CuentaBancaria(500)
cuenta2 = CuentaBancaria(300)

hilo1 = threading.Thread(target=cuenta1.transferir, args=(cuenta2, 200))
hilo2 = threading.Thread(target=cuenta2.transferir, args=(cuenta1, 100))

hilo1.start()
hilo2.start()

hilo1.join()
hilo2.join()

print(f"Saldo cuenta1: {cuenta1.saldo}")
print(f"Saldo cuenta2: {cuenta2.saldo}")

Saldo cuenta1: 400
Saldo cuenta2: 400
